<a href="https://colab.research.google.com/github/khyejin1231/Masters-Thesis/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls  # Check if required cuda 9.0 amd64-deb file is downloaded
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0
!nvcc --version
!pip install thundersvm

--2022-05-11 07:49:37--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2022-05-11 07:49:37--  https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?qeu_-2B9jokjl003N5XhizlXm_xfvk8zcd8G7bh4PLPEXMjrK2Zodzmw6JUQFs4HoUpoVHYVhYK6xKnlczhacqRY0sd5p

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from thundersvm import SVC

In [3]:
import pandas as pd
from google.colab import drive
import os
drive.mount('/content/drive', force_remount = True)
PATH = '/content/drive/My Drive/Colab Notebooks/BDS_Thesis'

#!pip install pickle5
#import pickle5 as pickle

#infile = open(PATH+'/final_labels_no_artists_lemma_10.pkl','rb')
#df = pickle.load(infile)


#url = 'https://github.com/khyejin1231/Masters-Thesis/blob/main/final_labels_no_artists_lemma_10.pkl?raw=true'

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/BDS_Thesis/joined_selected_BERT_xartist_lemma10.csv')

df.reset_index(drop=True, inplace=True)
df = df.sample(n=30000)

Mounted at /content/drive


In [4]:
df = df.sample(frac=1)
df.columns
X = df.iloc[:, 1:769]
y = df.iloc[:,-1]


In [5]:
#Spliting train and test
X_1, X_test, y_1, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [6]:
#Spliting train and valid
X_train, X_val, y_train, y_val = train_test_split(X_1, y_1, test_size=0.1, stratify=y_1)

In [8]:
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.metrics import f1_score
from scipy import stats
import numpy as np

rand_list = {"C": stats.uniform(0,1000),
             "gamma": stats.uniform(0,8)}
              
rand_search = RandomizedSearchCV(SVC(), param_distributions = rand_list, n_iter = 30, cv = 3, n_jobs=-1, random_state = 2022, scoring = 'f1') 
rand_search.fit(X_train, y_train) 

RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=30, n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82a962d10>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc82a962e90>},
                   random_state=2022, scoring='f1')

In [9]:
rand_search.cv_results_

{'mean_fit_time': array([16.05457791, 17.01357476, 16.02125637, 15.57331991, 16.03820133,
        15.67763797, 15.85279075, 15.36373067, 15.89392066, 16.02172184,
        16.29692896, 15.12025015, 15.08262817, 16.25272242, 15.93339992,
        16.37884537, 20.54822771, 17.12463633, 15.86862453, 15.34497849,
        18.22986118, 16.95103534, 15.66234223, 15.40831796, 15.56246988,
        15.58675822, 16.19005656, 16.11167606, 91.86239584, 16.38840214]),
 'mean_score_time': array([2.57703837, 2.08894142, 2.01654402, 2.33378394, 1.97724771,
        2.27288119, 2.03875383, 2.07928499, 2.00671101, 2.55862149,
        2.87454891, 2.23251216, 1.95792015, 2.21917041, 2.26143177,
        2.28216902, 2.17022943, 2.06730851, 2.0832363 , 2.33752346,
        2.33778866, 2.27883744, 1.98807494, 2.23089004, 1.95174042,
        2.13885307, 2.00124423, 1.99599989, 2.14042966, 2.5483884 ]),
 'mean_test_score': array([0.05711438, 0.50294066, 0.05839654, 0.05729879, 0.05711881,
        0.05674684, 0.05581

In [12]:
rand_search.best_score_

0.5132121565038973

In [ ]:
clf = SVC(verbose=True, gamma=0.5, C=100)
clf.fit(X_train,y_train)
y_predict=clf.predict(X_val)
tn, fp, fn, tp = confusion_matrix(y_val, y_predict).ravel()
f1 =f1_score(y_val, y_predict, average=None)

In [ ]:
f1 = []
gamma = np.logspace()
C = np.logspace()
 

clf = SVC(verbose=True, gamma=0.5, C=100)
clf.fit(X_train,y_train)
y_predict=clf.predict(X_val)
tn, fp, fn, tp = confusion_matrix(y_val, y_predict).ravel()
f1 =f1_score(y_val, y_predict, average=None)
